**Autores**: 
1. Caio Fernandes - 170138798
2. Lucas Dutras - 170050939

Este projeto tem como objetivo utilizar técnicas de data science juntamente com o conteúdo de **Percorrer Grafos** para indicar qual o caminho mais rápido para chegar em um país a partir de um país inicial.

Utilizaremos para isto um dataset disponibilizado no Kaggle com informações sobre [Cidades e Fronteiras](https://www.kaggle.com/resheto/country-borders).

In [2]:
import pandas as pd

In [61]:
df = pd.read_csv('./data/border_info.csv')
df = df.sort_values(by=['country_name'])

## Avaliação do Dado

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           728 non-null    int64 
 1   country_name         728 non-null    object
 2   country_border_name  642 non-null    object
dtypes: int64(1), object(2)
memory usage: 17.2+ KB


## Limpando o dado 

Removendo países sem fronteira

In [50]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
rows_with_NaN

,Unnamed: 0,country_name,country_border_name
41,41,Aland Islands,NaN
30,30,American Samoa,NaN
11,11,Anguilla,NaN
24,24,Antarctica,NaN
10,10,Antigua and Barbuda,NaN
...,...,...,...
684,684,United States Minor Outlying Islands,NaN
704,704,Vanuatu,NaN
699,699,Virgin Islands (British),NaN
700,700,Virgin Islands (U.S.),NaN


Temos 86 países sem fronteiras no dado

Removendo-os:

In [51]:
df = df.drop(rows_with_NaN.index)

Notamos que, neste dado em questão, temos 727 valores.
Tal dado também apresenta duas colunas, uma `country_name` representando o país e outra chamada `coutry_border_name` que representa um fronteira daquele país.

In [79]:
df.head()

,Unnamed: 0,country_name,country_border_name
0,0,Andorra,France
1,1,Andorra,Spain
2,2,United Arab Emirates,Oman
3,3,United Arab Emirates,Saudi Arabia
4,4,Afghanistan,China


Logo percebemos um problema, a maneira como o dado está organizado, nos trás informações duplicadas sobre países.
Para corrigir este problema, faremos, para cada país, uma coluna com uma lista de todas as fronteiras.
Com isso, teremos na estrutura do `DataFrame`, países únicos na coluna `country_name` com suas fronteiras listada na coluna `border_list`.

In [52]:
k = df.groupby('country_name')['country_border_name'].apply(lambda x: list(x))

In [55]:
df = df.drop(columns=['country_border_name'])

In [56]:
df = df.drop_duplicates(subset=['country_name'])

In [57]:
aux = [item for item in k]

In [58]:
df['border_list'] = aux

In [59]:
df.head(25)

,Unnamed: 0,country_name,border_list
9,9,Afghanistan,"[Uzbekistan, China, Iran, Pakistan, Tajikistan..."
15,15,Albania,"[Serbia, North Macedonia, Montenegro, Greece]"
200,200,Algeria,"[Mali, Tunisia, Niger, Libya, Morocco, Western..."
0,0,Andorra,"[France, Spain]"
20,20,Angola,"[Congo (Kinshasa), Congo (the Democratic Repub..."
29,29,Argentina,"[Uruguay, Brazil, Bolivia, Chile, Paraguay]"
17,17,Armenia,"[Georgia, Iran, Turkey, Azerbaijan]"
37,37,Austria,"[Slovenia, Czechia, Germany, Hungary, Liechten..."
43,43,Azerbaijan,"[Georgia, Turkey, Armenia, Iran, Russia]"
52,52,Bangladesh,"[Myanmar, India]"
